# 1. Environment Setup

Actions performed:
1. Sets up & tests connectivity to Azure OpenAI
2. Defines functions to call AzureOpenAI

In [ ]:
import os
import openai
from urllib.parse import urlparse
import dotenv


# Load environment variables from .env file
dotenv_path = "."
dotenv.load_dotenv(dotenv_path)
print("Azure OpenAI Deployment: ", os.getenv('DEPLOYMENT_NAME'))

import os
openai.api_type = "azure"
openai.api_version = os.getenv("API_VERSION")
openai.api_base = os.getenv("ENDPOINT")  # Your Azure OpenAI resource's endpoint value.
openai.api_key = os.getenv("API_KEY")

# function below calls Azure OpenAI and returns a text response
def run_chat_completion(prompt_template):
    resp = openai.ChatCompletion.create(
        engine=os.getenv('DEPLOYMENT_NAME'),
        messages=[
            {"role": "system", "content": "You're a helpful analyst."},
            {"role": "user", "content": prompt_template}
        ]
    )

    return resp

# function below calls Azure OpenAI and returns a JSON response - note you also need to ask for JSON formatting in your prompt
def run_chat_completion_json(prompt_template):
    resp = openai.ChatCompletion.create(
        engine=os.getenv('DEPLOYMENT_NAME'),
        response_format={ "type": "json_object" },
        messages=[
            {"role": "system", "content": "You're a helpful analyst."},
            {"role": "user", "content": prompt_template}
        ]
    )

    return resp


# Example usage:
new_prompt_template = "What is Australia like for food in 10 words or less?"
response = run_chat_completion(new_prompt_template)


print(response["choices"][0]["message"]["content"])

### Step 2: Import Data

Source: https://www.kaggle.com/datasets/parve05/customer-review-dataset/
1. Sign up for a free Kaggle account if you haven't already
2. Download the dataset (it's in a zip file)
3. Extract the file from the zip and save it in this folder. The datafile was called "redmi6.csv" when I last accessed, just note the name and if it is different update the code below so it can be found by the script

In [ ]:
import csv

file_path = 'redmi6.csv'
data = []
feedback = []

with open(file_path, 'r', encoding='utf-8', errors='replace') as csvfile:
    csvreader = csv.reader(csvfile)

    for row in csvreader:
        # print(row)
        data.append(row)
        feedback.append(row[5])


print("CSV file loaded")

### Step 3: Import Feedback data

Actions:
1. Loads the Kaggle data into memory
2. Prints out the first two rows to demonstrate what is in the dataset

In [ ]:
print("Total rows of data:", len(feedback))
print("First 2 feedback items:")
print(feedback[:2])

### Step 4. Understand sentiment based on feedback

Now let's ask GPT4 to read summarise what the reviewers thought about the mobile phone.

In [ ]:
new_prompt_template = f"Using the feedback below what is the general sentiment? {feedback}"
response = run_chat_completion(new_prompt_template)


print(response["choices"][0]["message"]["content"])

#### Step 5. Categorise the feedback

A paragraph or two is good, but as humans often we like to group information to better understand it. Let's ask GPT4 to instead summarise the information and group it.

In [ ]:
new_prompt_template = f"Using the feedback below what are the main categories it falls into? Provide these as a list, grouped in positive/neutral/negative sentiment. Feedback: {feedback}"
response = run_chat_completion(new_prompt_template)


print(response["choices"][0]["message"]["content"])

#### Step 6. Make the feedback understandable by an app

One of the amazing things GPT4 can do is turn QUALitative data (such as the results above) into QUANTitative data. This is incredibly powerful, as it enables us to build the output of a document analysis into a workflow.

For example, you could review a supplier contract to confirm if there is a limitation of liability clause in there. If there isn't one, GPT4 could return "false", and you could then build a workflow that requests the supplier to amend the contract and ensure they include the appropriate clause.

OpenAI have built a specific capability we can leverage is the "response_format={ "type": "json_object" } argument. This instructs the LLM to return a JSON-compliant response which apps can then use.

In [ ]:
new_prompt_template = f"Using the feedback below what are the main themes shared? Respond with a JSON array, with each theme a key value pair, capturing the general sentiment of that theme as the value (positive/neutral/negative). {feedback}."
response = run_chat_completion_json(new_prompt_template)


print(response["choices"][0]["message"]["content"])